## Installing packages

In [18]:
# !pip install deeppavlov

In [19]:
# !python -m deeppavlov install ner_ontonotes_bert

## Downloading models

You need manually download to colab this file: https://docs.google.com/uc?export=download&id=1mQRioZ6tZBSqyOcWOEcabhgV975r_x7d (due to unique confirmation code)

In [20]:
# Code from https://stackoverflow.com/questions/48735600/file-download-from-google-drive-to-colaboratory
# append the id to this REST command
# file_download_link = 'https://docs.google.com/uc?export=download&id=1mQRioZ6tZBSqyOcWOEcabhgV975r_x7d'
# !wget -O deeppavlov.zip --no-check-certificate "$file_download_link"
# Not working because file is too large

In [21]:
!unzip deeppavlov.zip

Archive:  deeppavlov.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of deeppavlov.zip or
        deeppavlov.zip.zip, and cannot find deeppavlov.zip.ZIP, period.


In [22]:
!cp -r .deeppavlov ~/.deeppavlov

cp: cannot stat '.deeppavlov': No such file or directory


## Importing model

In [23]:
from deeppavlov import configs, build_model, train_model
from copy import deepcopy
import os

In [24]:
# ner_model = build_model(configs.ner.conll2003_m1, download=True)

## Train

If you want to reproduce traning it tooks about 8h (4h for a model) on the one 1080ti

In [25]:
# # Note: you need to unzip deeppavlov.zip first, because the dataset
# ner_model_bert = build_model(configs.ner.ner_conll2003_bert, download=True)
# !python -m deeppavlov train ner_conll2003_bert
# ner_model_bert = build_model(configs.ner.ner_conll2003_torch_bert, download=True)
# !python -m deeppavlov train ner_conll2003_torch_bert

## Tensorflow model

In [26]:
ner_model_bert = build_model(configs.ner.ner_conll2003_bert, download=False)

2020-12-06 23:31:16.982 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/x11dpg-qt/.deeppavlov/models/ner_conll2003_bert/tag.dict]
2020-12-06 23:31:42.646 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/x11dpg-qt/.deeppavlov/models/ner_conll2003_bert/model]


INFO:tensorflow:Restoring parameters from /home/x11dpg-qt/.deeppavlov/models/ner_conll2003_bert/model


### Try on the train data

In [27]:
ner_model_bert(["Windows is greater than macos"])

[[['Windows', 'is', 'greater', 'than', 'macos']],
 [['B-Object', 'O', 'B-Predicate', 'O', 'B-Object']]]

### Functions to repair tokenizition

In [28]:
def change_two_symbols(symbol, bad_preds):
    preds = deepcopy(bad_preds)
    mask = [not symbol == text for text in text_list[:-1]]
    new_mask = deepcopy(mask)
    n_insertion = 0
    for i, mask_number in enumerate(mask):
        if mask_number==False:
            new_mask[i+n_insertion] = True
            new_mask.insert(i+1+n_insertion, False)
            n_insertion += 1
    mask = new_mask
    for i, mask_number in enumerate(mask):
        if mask_number==False:
            p = preds[1][0][i]
            t = preds[0][0][i]
            print(i, p, t)
    new_p = [[[]],
            [[]]]
    for m, p0, p1 in zip(mask, preds[0][0], preds[1][0]):
        if m:
            new_p[0][0].append(p0)
            new_p[1][0].append(p1)
        else:
            print(m, p0, p1)
    return new_p
def change_three_symbols(symbol, bad_preds):
    preds = deepcopy(bad_preds)
    mask = [not symbol == text for text in text_list[:-1]]
    new_mask = deepcopy(mask)
    n_insertion = 0
    for i, mask_number in enumerate(mask):
        if mask_number==False:
            new_mask[i+n_insertion] = True
            new_mask.insert(i+1+n_insertion, False)
            n_insertion+=1
            new_mask.insert(i+1+n_insertion, False)
            n_insertion+=1
    mask = new_mask
    for i, mask_number in enumerate(mask):
        if mask_number==False:
            p = preds[1][0][i]
            t = preds[0][0][i] 
            print(i, p, t)
    new_p = [[[]],
            [[]]]
    for m, p0, p1 in zip(mask, preds[0][0], preds[1][0]):
        if m:
            new_p[0][0].append(p0)
            new_p[1][0].append(p1)
        else:
            print(m, p0, p1)
    return new_p

In [29]:
two_symbols = (
    '),',
    ').',
    '++',
    ':)',
    '--',
    '.,',
    '.)',
    '!!',
    '):',
    '.:',
    '%.',
    '..'
    
)
three_symbols = (
    '++.',
    '.),',
    '++,',
    '(!)',
    '...',
    '///',
)


### Remove file with answers if exist

In [30]:
try:
    os.remove("test_with_answers_t.tsv")
except OSError:
    pass

In [31]:
with open("test_with_answers_t.tsv", 'a+') as f_out:
    with open("test_no_answers.tsv", 'r') as f_in:
        text_list = []
        for k, line in enumerate(f_in.readlines()):
            line = line.replace('\n', "")
            text_list.append(line)
            if line == '':
                preds = ner_model_bert([" ".join(text_list)])
                if k==7657:
                    print(text_list)
                    print(preds[0][0].pop(12), preds[1][0].pop(12))
                    print(preds[0][0].pop(21), preds[1][0].pop(21))
                    print(preds[0][0].pop(26), preds[1][0].pop(26))
                else:
                    for s2 in two_symbols:
                        if s2 in text_list:
                            print(k, "two symbols error")
                            preds = deepcopy(change_two_symbols(symbol=s2, bad_preds=preds))
                    for s3 in three_symbols:
                        if s3 in text_list:
                            print(k, "three symbols error")
                            preds = change_three_symbols(symbol=s3, bad_preds=preds)
                if k==5612:
                    preds[0][0].pop(13)
                    preds[1][0].pop(13) 
                elif len(preds[0][0]) != len(text_list[:-1]):
                    print(k)
                    print([a==b for a,b in zip(text_list[:-1], preds[0][0])].index(False))
                    print(text_list[:-1])
                    print(preds[0][0])
                    print(preds[1][0])    
                for word, pred in zip(text_list[:-1], preds[1][0]):
                    if pred == "O-smt" or pred == "O-pnk":
                        pred = "O"
                    answer = f"{word}\t{pred}\n"
                    print(k, answer)
                    f_out.write(answer)
                f_out.write("\n")
                text_list = []

23 plus	O

23 ,	O

23 android	B-Object

23 is	O

23 developing	B-Aspect

23 a	O

23 way	O

23 faster	B-Predicate

23 than	O

23 ios	O

23 so	O

23 it	O

23 has	O

23 chances	O

23 to	O

23 become	O

23 a	O

23 laptop	O

23 replacement	O

23 earlier	B-Predicate

23 than	O

23 ios	O

23 .	O

39 went	O

39 to	O

39 android	O

39 earlier	O

39 this	O

39 year	O

39 after	O

39 being	O

39 convinced	O

39 its	O

39 better	B-Predicate

39 then	O

39 ios	O

39 apple	B-Object

39 .	O

71 the	O

71 version	O

71 we	O

71 showed	O

71 here	O

71 is	O

71 ios	O

71 only	O

71 ,	O

71 because	O

71 the	O

71 ios	O

71 code	O

71 supported	B-Aspect

71 ibeacons	I-Aspect

71 earlier	B-Predicate

71 than	O

71 android	O

71 ,	O

71 but	O

71 we	O

71 are	O

71 almost	O

71 finished	O

71 with	O

71 an	O

71 android	O

71 one	O

71 as	O

71 well	O

71 .	O

96 he	O

96 '	O

96 ll	O

96 quickly	O

96 realize	O

96 why	O

96 android	B-Object

96 is	O

96 a	O

96 better	B-Predicate

96 os	O

96 -	O

96 an

## Pytorch

In [32]:
ner_model_bert = build_model(configs.ner.ner_conll2003_torch_bert, download=False)

2020-12-06 23:34:58.90 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/x11dpg-qt/.deeppavlov/models/ner_conll2003_torch_bert/tag.dict]
2020-12-06 23:35:15.446 INFO in 'deeppavlov.models.torch_bert.torch_bert_sequence_tagger'['torch_bert_sequence_tagger'] at line 358: Load path /home/x11dpg-qt/.deeppavlov/models/ner_conll2003_torch_bert/model is given.
2020-12-06 23:35:15.449 INFO in 'deeppavlov.models.torch_bert.torch_bert_sequence_tagger'['torch_bert_sequence_tagger'] at line 365: Load path /home/x11dpg-qt/.deeppavlov/models/ner_conll2003_torch_bert/model.pth.tar exists.
2020-12-06 23:35:15.449 INFO in 'deeppavlov.models.torch_bert.torch_bert_sequence_tagger'['torch_bert_sequence_tagger'] at line 366: Initializing `TorchBertSequenceTagger` from saved.
2020-12-06 23:35:15.450 INFO in 'deeppavlov.models.torch_bert.torch_bert_sequence_tagger'['torch_bert_sequence_tagger'] at line 369: Loading weights from /home/x11dpg-qt/.deeppavlov

### Try on the train data

In [33]:
ner_model_bert(["Windows is greater than macos"])

[[['Windows', 'is', 'greater', 'than', 'macos']],
 [['B-Aspect', 'O', 'B-Predicate', 'O', 'B-Object']]]

In [34]:
try:
    os.remove("test_with_answers_p.tsv")
except OSError:
    pass

In [35]:
with open("test_with_answers_p.tsv", 'a+') as f_out:
    with open("test_no_answers.tsv", 'r') as f_in:
        text_list = []
        for k, line in enumerate(f_in.readlines()):
            line = line.replace('\n', "")
            text_list.append(line)
            if line == '':
                preds = ner_model_bert([" ".join(text_list)])
                if k==7657:
                    print(text_list)
                    print(preds[0][0].pop(12), preds[1][0].pop(12))
                    print(preds[0][0].pop(21), preds[1][0].pop(21))
                    print(preds[0][0].pop(26), preds[1][0].pop(26))
                else:
                    for s2 in two_symbols:
                        if s2 in text_list:
                            print(k, "two symbols error")
                            preds = deepcopy(change_two_symbols(symbol=s2, bad_preds=preds))
                    for s3 in three_symbols:
                        if s3 in text_list:
                            print(k, "three symbols error")
                            preds = change_three_symbols(symbol=s3, bad_preds=preds)
                if k==5612:
                    preds[0][0].pop(13)
                    preds[1][0].pop(13) 
                elif len(preds[0][0]) != len(text_list[:-1]):
                    print(k)
                    print([a==b for a,b in zip(text_list[:-1], preds[0][0])].index(False))
                    print(text_list[:-1])
                    print(preds[0][0])
                    print(preds[1][0])    
                for word, pred in zip(text_list[:-1], preds[1][0]):
                    if pred == "O-smt" or pred == "O-pnk":
                        pred = "O"
                    answer = f"{word}\t{pred}\n"
                    print(k, answer)
                    f_out.write(answer)
                f_out.write("\n")
                text_list = []

23 plus	O

23 ,	O

23 android	O

23 is	O

23 developing	B-Aspect

23 a	O

23 way	O

23 faster	B-Predicate

23 than	O

23 ios	O

23 so	O

23 it	O

23 has	O

23 chances	O

23 to	O

23 become	O

23 a	O

23 laptop	O

23 replacement	O

23 earlier	B-Predicate

23 than	O

23 ios	O

23 .	O

39 went	O

39 to	O

39 android	O

39 earlier	O

39 this	O

39 year	O

39 after	O

39 being	O

39 convinced	O

39 its	O

39 better	B-Predicate

39 then	O

39 ios	B-Object

39 apple	B-Object

39 .	O

71 the	O

71 version	O

71 we	O

71 showed	O

71 here	O

71 is	O

71 ios	O

71 only	O

71 ,	O

71 because	O

71 the	O

71 ios	O

71 code	O

71 supported	O

71 ibeacons	B-Object

71 earlier	B-Predicate

71 than	O

71 android	B-Object

71 ,	O

71 but	O

71 we	O

71 are	O

71 almost	O

71 finished	O

71 with	O

71 an	O

71 android	O

71 one	O

71 as	O

71 well	O

71 .	O

96 he	O

96 '	O

96 ll	O

96 quickly	O

96 realize	O

96 why	O

96 android	B-Object

96 is	O

96 a	O

96 better	B-Predicate

96 os	B-Aspect

96 -	O

## Final step

Finnaly, you need to download test_with_answers_p.tsv or test_with_answers_t.tsv on your computer, zip them and submit to codalab: https://competitions.codalab.org/competitions/27655#participate-submit_results